In [124]:
import pandas as pd

cars = pd.read_csv('cars_clean.csv')

In [125]:
test_car = {
    "manufacturer": "audi", #
    "model": "a3", #
    "airbag": "front-, seiten-airbags", #
    "climate_control": "klimaanlage", #
    "first_registration": 2017.0, # ranking
    "fuel": "benzin", #
    "color": "silber", #----------
    "condition": "beschädigt, unfallfahrzeug, nicht fahrtauglich", #--------------
    "gear": "automatik",  #
    "hu": "2022.0", # ---------
    "interior":"andere", #----------
    "mileage":128000.0, # --------------
    "num_of_owners":1.0, # ------------
    "num_seats":5.0, #
    "power":150.0, # ranking
    "cabrio":0.0, # ---------
    "kleinwagen":1.0, # --------
    "kombi":0.0, # -------
    "limousine":0.0, # -----
    "sportwagen":0.0, # -----
    "gelaendewagen":0.0, # -----
    "van":0.0, # -----
    "co2":114.0, # ranking
    "consumption":5, # ranking
    "cubicCapacity":1395.0, # ranking
    "doors":4.0, #
    "emission_class":6.0, #
    "environment_class":4.0 #
}

In [126]:
for key in ["first_registration", "power", "cubicCapacity", "consumption", "co2"]:
    min_ = cars[key].quantile(0.01)
    max_ = cars[key].quantile(0.99)
    cars[key+"_norm"] = (cars[key] - min_) / (max_ - min_)
    test_car[key+"_norm"] = (test_car[key] - min_) / (max_ - min_)

In [127]:
cars_filtered = cars[cars['title'].str.contains(test_car['model'].lower())]
for col in ["manufacturer", "fuel", "climate_control", "gear", "airbag", "environment_class", "emission_class", "doors", "num_seats"]:
    cars_filtered = cars_filtered[cars_filtered[col] == test_car[col]]
    break
cars_filtered.shape

(1543, 35)

In [128]:
from sklearn.metrics.pairwise import pairwise_distances

# calculate pairwise distances of the co2 and consumption columns to the test car
cars_filtered["distance"] = pairwise_distances(cars_filtered[["co2_norm", "consumption_norm","first_registration_norm", "power_norm", "cubicCapacity_norm"]],
                                               [[test_car["co2_norm"], test_car["consumption_norm"], test_car["first_registration_norm"], test_car["power_norm"], test_car["cubicCapacity_norm"]]],
                                               metric="euclidean")

In [129]:
# print the columns distance and co2 and consumption and sort by distance descending
cars_filtered[["distance", "co2", "consumption", "first_registration", "power", "cubicCapacity"]].sort_values(by="distance", ascending=True).head(10)

,distance,co2,consumption,first_registration,power,cubicCapacity
156526,0.011947,116.0,5.133333,2017,150.0,1395.0
40383,0.011947,116.0,5.133333,2017,150.0,1395.0
156550,0.011947,116.0,5.133333,2017,150.0,1395.0
42672,0.014383,110.0,4.933333,2017,150.0,1395.0
182387,0.016724,110.0,4.866667,2017,150.0,1395.0
159018,0.020831,112.0,4.733333,2017,150.0,1395.0
151009,0.020831,112.0,4.733333,2017,150.0,1395.0
143384,0.020831,112.0,4.733333,2017,150.0,1395.0
3140,0.020831,116.0,5.266667,2017,150.0,1395.0
182066,0.020905,119.0,5.166667,2017,150.0,1395.0


In [143]:
import webbrowser

webbrowser.open(cars.iloc[3140]["url"])



True

In [145]:
# ----------------------------------------------------------------------------------------------------------------------

import requests

urls = {
    "offline": "https://suchen.mobile.de/fahrzeuge/details.html?id=341475570&cn=DE&cn=DE&cn=DE&gn=K%C3%B6ln"
               "&isSearchRequest=true&ll=50.94%2C6.96&makeModelVariant1.makeId=1900&makeModelVariant1.modelId=8"
               "&pageNumber=1&rd=21&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&usage"
               "=USED&action=topInCategory&searchId=b26ebd9b-1733-782f-9c45-6ef27025bdb7",
    "no img": "https://suchen.mobile.de/fahrzeuge/details.html?id=341908370&cn=DE&cn=DE&cn=DE&gn=Kiel&isSearchRequest"
              "=true&ll=54.32%2C10.14&makeModelVariant1.makeId=1900&makeModelVariant1.modelId=8&pageNumber=1&rd=21"
              "&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&searchId=75d47ec9-e13e-593e"
              "-25ea-675331d42099",
    "online": "https://suchen.mobile.de/fahrzeuge/details.html?id=341934995&cn=DE&cn=DE&cn=DE&gn=Duisburg"
              "&isSearchRequest=true&ll=51.43%2C6.76&makeModelVariant1.makeId=1900&makeModelVariant1.modelId=8"
              "&pageNumber=1&rd=22&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&fnai=next"
              "&searchId=22fc764e-2efe-b294-80b0-9af354046bb0 "
}


def get_img(url):
    html = requests.get(url, headers={b'Accept': b'text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8',
                                      b'Accept-Language': b'en',
                                      b'User-Agent': b'Scrapy/2.6.1 (+https://scrapy.org/)',
                                      b'Accept-Encoding': b'gzip, deflate'}).text
    # print(html)

    if html.find("cycle-slide") != -1:
        img_url = html.split("cycle-slide")[1].split("src=\"")[1].split("\"")[0]
    elif html.find("vip-error__details__image") != -1:
        img_url = html.split("vip-error__details__image")[1].split("src=\"")[1].split("\"")[0]
        img_url_html = requests.get(img_url).text
        if img_url_html.find('"code":404') != -1:
            img_url = "https://upload.wikimedia.org/wikipedia/commons/b/b9/Nocar.svg"
    else:
        img_url = "https://upload.wikimedia.org/wikipedia/commons/b/b9/Nocar.svg"

    return img_url


for key in urls.keys():
    print(key, get_img(urls[key]), sep=": ")



offline: https://img.classistatic.de/api/v1/mo-prod/images/31/31782b15-ddcb-4cd4-98db-9380bb3f38e7?rule=mo-360.jpg
no img: https://upload.wikimedia.org/wikipedia/commons/b/b9/Nocar.svg
online: https://img.classistatic.de/api/v1/mo-prod/images/4c/4c1154d2-cf29-42d5-9c80-e3b7e424eb3b?rule=mo-1024.jpg
